# Подгрузка библиотек и данных

In [100]:
from utils.support import *

from sklearn import model_selection, ensemble, linear_model, tree, \
    naive_bayes, neighbors, preprocessing, cluster, mixture, svm, decomposition
from sklearn.metrics import roc_curve, recall_score, make_scorer
from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
from imblearn import over_sampling
from imblearn.pipeline import Pipeline

import lightgbm as lgbm
import catboost as cat

import warnings
warnings.filterwarnings('ignore')

In [101]:
data = open_data('data/bank_data_final_2.csv')
lin_features = open_list('logs/top_linear_features.txt')
tree_features = open_list('logs/top_non_linear_features.txt')
cat_features = open_list('logs/top_cat_features_list.txt')
kbest_features = open_list('logs/best_k_features.txt')

In [102]:
df_lin = data[lin_features]
df = data[tree_features]
new_list_rf = ['name_email_similarity',
 'current_address_months_count',
 'email_is_free',
 'bank_months_count',
 'keep_alive_session',
 'CA_AC',
 'payment_type_AC',
 'windows_AC',
 'windows_AB',
 'AC_BA',
 'AB_BA',
 'n_e_cat_similarity',
 'cur_address_months_binned_first_spike',
 'cur_address_fraud_zone',
 'cur_address_months_binned_end_spike',
 'is_credit_limit_high',
 'new_email_entrance']



In [103]:
data['fraud_bool'].value_counts(normalize=True)

fraud_bool
0    0.988971
1    0.011029
Name: proportion, dtype: float64

In [104]:
data.head(2)

,fraud_bool,income,name_email_similarity,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,zip_peak,bank_branch_count_8w,bank_branch_peak,zip_transaction_ratio,max_zip_sum,zip_sum_ratio,vel_1,vel_2,vel_3,vel_4
634102,0,7,0.546775,375,3,0.027343,-0.654816,AB,1455,3871.735367,...,1,14,1,194.000000,291000.0,7.275,-2114.180477,-1776.742716,107.040705,-0.011601
173005,0,8,0.715285,379,4,0.007504,18.068853,AA,1086,9501.114204,...,1,748,0,2.899866,217200.0,5.430,5708.831358,1203.137209,-235.726930,-0.124511


In [105]:
drop_list = []

# преобразование data для экономии памяти
for col in data.columns:
    # если признак имеет менее 128 уникальных значений
    if data[col].dtype == 'O':
        drop_list.append(col)
    elif data[col].nunique() < 128:
        # преобразуем формат
        data[col] = data[col].astype('uint8')
        
data.drop(drop_list, axis=1, inplace=True)

In [106]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 634102 to 471855
Data columns (total 79 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   fraud_bool                              750000 non-null  uint8  
 1   income                                  750000 non-null  uint8  
 2   name_email_similarity                   750000 non-null  float64
 3   current_address_months_count            750000 non-null  int64  
 4   customer_age                            750000 non-null  uint8  
 5   days_since_request                      750000 non-null  float64
 6   intended_balcon_amount                  750000 non-null  float64
 7   zip_count_4w                            750000 non-null  int64  
 8   velocity_6h                             750000 non-null  float64
 9   velocity_24h                            750000 non-null  float64
 10  velocity_4w                             7500

In [107]:
len(lin_features)

32

In [108]:
# tree_features

In [109]:
mm_scaler = preprocessing.MinMaxScaler((0, 1))
df_lin['proposed_credit_limit'] = mm_scaler.fit_transform(df_lin[['proposed_credit_limit']])
df['proposed_credit_limit'] = mm_scaler.fit_transform(df[['proposed_credit_limit']])
df['proposed_credit_limit']

634102    0.005236
173005    0.005236
40774     0.005236
606711    0.424084
429877    0.005236
            ...   
187743    0.005236
126160    0.418848
941678    0.005236
903476    0.005236
471855    0.685864
Name: proposed_credit_limit, Length: 750000, dtype: float64

In [110]:
scaler = preprocessing.RobustScaler()

features_to_scale_lin = [
    feature for feature in lin_features
    if feature not in cat_features
]

features_to_scale_tree = [
    feature for feature in tree_features
    if feature not in cat_features
]

df_lin[features_to_scale_lin] = scaler.fit_transform(df_lin[features_to_scale_lin])
df[features_to_scale_tree] = scaler.fit_transform(df[features_to_scale_tree])

Что ж, данные почти отмасштабированы, осталась одна проблема - дисбаланс классов. Его будем решать оверсемплингом с помощью алгоритма ADASYN, т.к. в процессе изучения данных обнаружили, что довольно часто примеры пересекаются между собой, а значит, нужно создавать больше примеров на границе классов.
<br>Но загвоздка в том, что хотелось использовать кросс-валидацию, т.к. данных не так много. Поэтому обучать будем пайплайн, а его первый шаг для удобства сохраним:

In [111]:
# ada = ('ADASYN', over_sampling.ADASYN(n_neighbors=80, random_state=rs))

*P.S. хотел сделать все очень новомодно: кроссвалидацию с пайплайном, в котором сначала данные оверсемплируются ADASYN, а затем применяется уже модель и по ней происходит GridSearchCV, но это чоень затратно по времени: обычная логистическая регрессия обучалась более 15 минут. Поэтому просто создадим валидационную выборку и сразу же к ней применим ADASYN:*

In [112]:
# разделяем выборку для линейных моделей на тренировочную и валидационную
X_train_lin, X_val_lin, y_train_lin, y_val_lin = train_test_split(df_lin, data['fraud_bool'], 
                                                  test_size=0.3,
                                                  stratify=data['fraud_bool'], 
                                                  random_state=rs)
# разделяем выборку для деревьев на тренировочную и валидационную
X_train, X_val, y_train, y_val = train_test_split(df, data['fraud_bool'], 
                                                  test_size=0.3,
                                                  stratify=data['fraud_bool'], 
                                                  random_state=rs)
X_train.shape

(525000, 32)

In [113]:
X_train_lin

,credit_risk_score,max_account_sum,max_zip_sum,cur_address,n_e_norm_similarity,zip_transaction_ratio,device_distinct_emails_8w,birth_distinct_emails_logged,zip_sum_ratio,intended_balcon_amount,...,payment_type_AA,device_os_windows,device_os_linux,device_os_other,cur_address_months_binned_danger_spike,intended_balcon_peak_1,bank_branch_peak,income,proposed_credit_limit,BA_windows
135932,0.212766,-0.221088,0.341147,-0.212345,-0.392089,0.544353,0.0,-1.321928,-0.334191,3.883488,...,1,0,1,0,1,1,0,6,0.162304,0
989262,-0.074468,0.564627,-0.515228,0.613348,0.351951,-0.230274,0.0,-0.736966,-0.525635,-0.043334,...,0,1,0,0,1,0,1,8,0.005236,0
952350,1.489362,0.126532,0.244521,0.463542,-0.038725,-0.179410,0.0,-0.321928,-0.676333,0.039174,...,0,1,0,0,1,0,1,7,0.418848,1
649430,0.829787,-0.102040,0.657659,0.694247,0.055866,-0.242091,0.0,0.137504,-0.712039,0.055657,...,0,0,0,0,1,0,0,5,0.685864,0
167870,-0.712766,0.517008,-0.313974,0.698521,-1.347515,-0.163581,0.0,0.678072,0.066803,-0.020554,...,0,0,1,0,1,0,1,8,0.005236,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964248,1.010638,5.850341,1.996865,0.622733,0.126343,-0.067999,0.0,-0.152003,-0.472017,-0.040787,...,0,0,0,1,1,0,1,0,0.424084,0
4425,-0.606383,-0.292516,-0.171245,0.584535,-0.959653,0.032397,0.0,1.263034,0.486959,-0.069029,...,1,0,0,1,1,0,0,4,0.005236,0
158743,-0.819149,0.421770,-0.183786,1.110188,0.632191,-0.242316,0.0,0.584963,0.450042,-0.016127,...,1,0,0,1,0,0,0,0,0.005236,0
251481,-1.372340,0.421770,0.616154,-0.308187,-0.026334,-0.269932,0.0,0.263034,2.804851,-0.105998,...,0,0,0,1,1,0,0,4,0.005236,0


In [114]:
# применяем к ним ADASYN
adasyn = over_sampling.ADASYN(n_neighbors=32)
X_train_lin_resampled, y_train_lin_resampled = adasyn.fit_resample(X_train_lin, y_train_lin)
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)

In [115]:
X_train_resampled.shape

(1038580, 32)

In [116]:
X_train_lin_resampled.shape

(1037315, 32)

(Жесть, теперь понятно, почему обучение так много времени занимало)

Последний шаг - выбор метрики. Компания feedzai предложила следующий подход: считаем ROC кривую, находим на этой кривой точку со значением FPR 5%, берем ее как пороговое значение и так бинаризуем результат. Напишем функцию под это:

In [117]:
# функция, возвращающая recall для данного fpr
def get_recall(y_true, y_pred, target_fpr=0.05, is_probabilities_in=True):
    classes = np.unique(y_true)
    if len(classes) < 2:
        return 0.0

    try:
        fpr, tpr, thresholds = roc_curve(y_true, y_pred[:, 1])
        if len(fpr) == 0:
            return 0.0

        idx = np.abs(fpr - target_fpr).argmin()
        thresh = thresholds[idx]
        if is_probabilities_in:
            y_pred = (y_pred[:, 1] >= thresh).astype('uint8')

        return recall_score(y_true, y_pred)
    except Exception:
        return 0.0

recall_at_5_fpr = make_scorer(get_recall)

Наконец - к моделям!

# 4. Построение модели (baseline)

## 1. LogReg

Итак, начинаем от базовой логистической регрессии:

In [19]:
log_reg = linear_model.LogisticRegression(random_state=rs, max_iter=1200, solver='saga', penalty='l2',
                                          C=0.5)
log_reg.fit(X_train_lin, y_train_lin)
y_pred = log_reg.predict_proba(X_val_lin)
print('Recall for best LogReg: {} %'\
    .format(round(get_recall(y_val_lin, y_pred)*100)))

Recall for best LogReg: 42 %


Попробовал разные параметры и обучающие выборки. Результат с оверсемплингом постоянная: 33% (ADASYN показал себя лучше, чем SMOTE). А вот если обучать на обычных данных, то получается 42%. В любом случае, довольно низкий показатель, попробуем решающее дерево!

*P.S. ниже пытался подобрать гиперпараметры с помощью GridSearchCV, но резульатт был довольно времязатратный.*

In [ ]:
# # базовая логистическая регрессия
# log_reg = linear_model.LogisticRegression(random_state=42, max_iter=1200, solver='saga')
# # список параметров, которые нужно перебрать
# param_grid = {
#     'penalty': ['l1', 'l2'],
#     # # учитываем, что у нас много образцов, два клаcса и 32 фичи
#     # 'solver': ['saga', 'lbfgs'],
#     'tol': [1e-4, 1e-5],
#     'C': [1, 2, 0.5, 0.25] 
# }
# # # пайплайн с оверсемплингом
# # pipe = Pipeline([
# #     ada,
# #     ('lr_model', log_reg)
# # ])
# # поиск параметров
# grid_search = model_selection.RandomizedSearchCV(
#     log_reg, param_grid, scoring=recall_at_5_fpr, cv=3,
#     verbose=2, n_iter=8, error_score='raise'
# )

# grid_search.fit(X_train_lin_resampled, y_train_lin_resampled)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END .........................C=1, penalty=l2, tol=1e-05; total time=  42.1s
[CV] END .........................C=1, penalty=l2, tol=1e-05; total time=  35.1s
[CV] END .........................C=1, penalty=l2, tol=1e-05; total time=  35.6s
[CV] END .........................C=2, penalty=l1, tol=1e-05; total time=  45.4s
[CV] END .........................C=2, penalty=l1, tol=1e-05; total time=  40.3s
[CV] END .........................C=2, penalty=l1, tol=1e-05; total time=  40.5s
[CV] END .....................C=0.25, penalty=l1, tol=0.0001; total time=  34.0s
[CV] END .....................C=0.25, penalty=l1, tol=0.0001; total time=  32.1s
[CV] END .....................C=0.25, penalty=l1, tol=0.0001; total time=  31.3s
[CV] END .....................C=0.25, penalty=l2, tol=0.0001; total time=  32.3s
[CV] END .....................C=0.25, penalty=l2, tol=0.0001; total time=  28.1s
[CV] END .....................C=0.25, penalty=l2,

RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=LogisticRegression(max_iter=1200, random_state=42,
                                                solver='saga'),
                   n_iter=8,
                   param_distributions={'C': [1, 2, 0.5, 0.25],
                                        'penalty': ['l1', 'l2'],
                                        'tol': [0.0001, 1e-05]},
                   scoring=make_scorer(get_recall, response_method='predict'),
                   verbose=2)

In [ ]:
print('Recall for best LogReg: {} %'\
    .format(round(get_recall(grid_search.best_estimator_, X_val_lin, y_val_lin)*100)))

Recall for best LogReg: 0 %


## 2. Decision Tree

Теперь переходим к деревьям, и заодно - более "информативному" датасету.
<br><br>В гиперпараметрах сразу очевидны две вези: class_weight нужно ставить на balanced (чтобы вес меньшего класса был больше), критерий информативности - log_loss или entropy, т.к. нас интересуют точные значения вероятностей для метрики recall_at_5_fpr.

In [31]:
decision_tree = tree.DecisionTreeClassifier(random_state=rs, class_weight='balanced', criterion='gini',
                                            max_depth=100, min_samples_split=250, min_impurity_decrease=0.0001)
decision_tree.fit(X_train_resampled, y_train_resampled)
y_pred = decision_tree.predict_proba(X_val)
print('Recall for best Decision Tree: {} %'\
    .format(round(get_recall(y_val, y_pred)*100)))

Recall for best Decision Tree: 25 %


Без оверсемплинга 35%, с ним - максимум 30.

*P.S. здесь перепробовал разные методы: и SMOTE с k_neighbours 18, 30 и 500, и ADASYN с теми же показателями - ничего не помогает. Синтетические данные приносят шум, который ухудшает метрику.*

In [ ]:
decision_tree = tree.DecisionTreeClassifier(random_state=rs, class_weight='balanced', criterion='entropy')

# # Создаём пайплайн
# pipe = Pipeline([
#     ('smote', over_sampling.SMOTE(random_state=rs)),
#     ('tree', tree.DecisionTreeClassifier(
#         random_state=rs, 
#         class_weight='balanced', 
#         criterion='entropy'
#     ))
# ])

# Обновите param_grid для пайплайна
param_grid = {
    'max_depth': np.arange(15, 500, 60),
    'min_samples_split': np.arange(40, 120, 20),
    'min_impurity_decrease': np.linspace(0.01, 0.1, 4)
}

grid_search = model_selection.RandomizedSearchCV(
    decision_tree, param_grid, scoring=recall_at_5_fpr, cv=3,
    verbose=2, n_iter=12, return_train_score=True, error_score='raise'
)

grid_search.fit(X_train_resampled, y_train_resampled)

In [ ]:
# tre = tree.DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
#                        max_depth=495, min_impurity_decrease=0.01,
#                        min_samples_split=40, random_state=42)

# tre.fit(X_train_resampled, y_train_resampled)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=495, min_impurity_decrease=0.01,
                       min_samples_split=40, random_state=42)

In [ ]:
# save_model(grid_search.best_estimator_, 'dtree_1')

In [ ]:
get_recall(tre, X_val, y_val)

Target FPR: 0.050, Closest FPR: 0.063, Threshold: 0.8519
Positive predictions at threshold: 14908 / 225000


0.3424657534246575

## 3. Другие алгоритмы

Т.к. в кибермошеннических атаках часто используются одни и те же действия и инструменты, есть мысль, что довольно точно себя покажет кластеризация.

Попробуем следующие алгоритмы: kmeans, методы на принципе "naive bayes" и SVM. 

### K-Nearest Neighbours

In [ ]:
k_neighbours = neighbors.KNeighborsClassifier(algorithm='brute', 
                                              weights='distance', 
                                              metric='euclidean',
                                              n_neighbors=55,
                                              p=3)
k_neighbours.fit(X_train_lin, y_train_lin)
# param_grid = {
#     'n_neighbors': list(range(3, 50, 10)),
#     'p': [1, 1.5, 2, 3, 4, 5]
# }

# grid_search = model_selection.RandomizedSearchCV(
#     estimator=k_neighbours, param_distributions=param_grid, n_iter=6,
#     scoring=recall_at_5_fpr, cv=2,
#     verbose=2, return_train_score=True, error_score='raise'
# )

k_neighbours.fit(X_train_lin, y_train_lin)

KNeighborsClassifier(algorithm='brute', metric='euclidean', n_neighbors=55, p=3,
                     weights='distance')

In [33]:
neigh_pred = k_neighbours.predict_proba(X_val_lin)

In [ ]:
print('Recall for best KNeigbours (p=1, n=5): {} %'\
     .format(round(get_recall(y_val_lin, neigh_pred)*100)))

Recall for best KNeigbours: 21 %


In [25]:
print('Recall for best KNeigbours (p=1, n=25): {} %'\
     .format(round(get_recall(y_val_lin, neigh_pred)*100)))

Recall for best KNeigbours (p=1, n=25): 30 %


In [32]:
print('Recall for best KNeigbours (p=2, n=55): {} %'\
     .format(round(get_recall(y_val_lin, neigh_pred)*100)))

Recall for best KNeigbours (p=2, n=55): 33 %


In [35]:
print('Recall for best KNeigbours (p=3, n=55): {} %'\
     .format(round(get_recall(y_val_lin, neigh_pred)*100)))

Recall for best KNeigbours (p=3, n=55): 33 %


Видим, что алгоритм показывает метрику на уровне решающего дерева. Попробуем другой алгоритм.

### Naive Bayes

In [77]:
naive_baye = naive_bayes.BernoulliNB()
naive_baye.fit(X_train_resampled, y_train_resampled)

cluster_pred = naive_baye.predict_proba(X_val)
#cluster_pred[:, 0], cluster_pred[:, 1] = cluster_pred[:, 1], cluster_pred[:, 0]
print('Recall for best Naive Bayes (Bernoulli distr): {} %'\
    .format(round(get_recall(y_val, cluster_pred)*100)))

Recall for best Naive Bayes (Bernoulli distr): 37 %


In [76]:
naive_baye = naive_bayes.GaussianNB()
naive_baye.fit(X_train_resampled, y_train_resampled)

cluster_pred = naive_baye.predict_proba(X_val)
#cluster_pred[:, 0], cluster_pred[:, 1] = cluster_pred[:, 1], cluster_pred[:, 0]
print('Recall for best Naive Bayes (Gaussian distr): {} %'\
    .format(round(get_recall(y_val, cluster_pred)*100)))

Recall for best Naive Bayes (Gaussian distr): 36 %


Как видим, байесовские классификаторы на распределении Бернулли и Гаусса показывают себя даже лучше одиночного дерева.

Давайте здесь же попробуем BayesianGaussianMixture - алгоритм из модуля mixture библиотеки sklearn, который подбирает кластеры по распределениям:

In [ ]:
bayes_gauss_mixture = mixture.BayesianGaussianMixture(
    n_components=2, covariance_type='tied', max_iter=300, 
    init_params='k-means++', random_state=rs
)

bayes_gauss_mixture.fit(X_train_resampled)
cluster_pred = bayes_gauss_mixture.predict_proba(X_val)
print('Recall for best Bayes-Gaussian mixture: {} %'\
    .format(round(get_recall(y_val, cluster_pred)*100)))

Recall for best Bayes-Gaussian mixture: 7 %


Лучший результат - на оверсемпле с ковариационной матрицей "tied' (т.е. распределения кластеров связаны, имеют одинаковую форму). В остальных случаях получался 0

### SVM

Последний - метод опорных векторов. Хотел использовать нелинейные версии, но они слишком длительны по времени.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

svm_linear = LinearSVC(C=0.2, class_weight='balanced', max_iter=1000, random_state=rs)
calibrated_svm = CalibratedClassifierCV(svm_linear, cv=5)
calibrated_svm.fit(X_train_resampled, y_train_resampled)

cluster_pred = calibrated_svm.predict_proba(X_val)
print('Recall (linear SVM): {:.2f} %'.format(get_recall(y_val, cluster_pred)*100))

Recall (linear SVM): 33.40 %


Результат на уровне дерева решений.

Итого - кластеризация в данном случае показывает себя на уровне одиночного дерева и логистической регрессии.

# 5. Использование продвинутых методов: ансамблевые модели

В ансамблиевых моделях будем подбирать гиперпараметры с помощью optuna:

In [21]:
import optuna

## Random Forest Classifier

Пару замечаний:
1. 

In [52]:
model = ensemble.RandomForestClassifier(
    criterion='log_loss', 
    max_depth=30, 
    class_weight='balanced', 
    min_samples_leaf=200, 
    min_impurity_decrease=0.001, 
    random_state=rs
)
model.fit(X_train_resampled, y_train_resampled)

rf_pred = model.predict_proba(X_val)
print('Recall for Random Forest: {:.2f} %'.format(get_recall(y_val, rf_pred)*100))

Recall for Random Forest: 35.01 %


In [ ]:
model.feature_importances_

In [27]:
def make_optuna_objective(model, param_grid, X_train, y_train, X_val, y_val):
    """Функция для определения оптимизирующей функции для optuna

    Аргументы:
        model (): любая модель, поддерживающая sklearn API, в частности методы fit и fit_predict
        param_grid (dict): список параметров
        X_train (np.array): обучающая тренировочная выборка
        y_train (np.array): обучающая тренировочная целевая переменная
        X_val (np.array): валидационная тренировочная выборка
        y_val (np.array): валидационная тренировочная целевая переменная
    """
    # функция для подбора параметров для optuna
    def suggest(trial, name, space):
        if isinstance(space, tuple):
            if len(space) == 3: 
               # int пространство: (low, high, step) 
               return trial.suggest_int(name, *space) 
            elif isinstance(space[0], float): 
                # float пространство: (low, high)
                return trial.suggest_float(name, *space)
        elif isinstance(space, list):
            # пространство категориальных
            return trial.suggest_categorical(name, space)
        # для фиксированных значений
        else:
            return space
        
    # оптимизирующая функция
    def optuna_optimize_rf(trial):
        # сетка параметров
        param_grid_trial = {
            key: suggest(trial, key, space)
            for key, space in param_grid.items()
        }
        
        # обучаем модель
        model_train = model(**param_grid_trial)
        model_train.fit(X_train, y_train)
        # получаем показатель метрики для данного trial
        score = get_recall(y_val, model_train.predict_proba(X_val))
        
        return score
    
    return optuna_optimize_rf

In [ ]:
param_grid_rf = {
    'n_estimators': (150, 350, 50),
    'max_depth': (35, 55, 10),
    'min_samples_leaf':  (100, 400, 40),
    'min_impurity_decrease': (0.0005, 0.0008),
    'class_weight': 'balanced',
    'random_state': rs,
    'criterion': 'log_loss'
}

rf_optuna = make_optuna_objective(
    model=ensemble.RandomForestClassifier,
    param_grid=param_grid_rf,
    X_train=X_train_resampled,
    y_train=y_train_resampled,
    X_val=X_val,
    y_val=y_val
)

study_rf = optuna.create_study(study_name='Random Forest Classifier', direction='maximize')
study_rf.optimize(rf_optuna, n_trials=10)

[I 2025-08-07 14:31:51,167] A new study created in memory with name: Random Forest Classifier
[I 2025-08-07 14:34:31,189] Trial 0 finished with value: 0.3311845286059629 and parameters: {'n_estimators': 150, 'max_depth': 45, 'min_samples_leaf': 140, 'min_impurity_decrease': 0.01800050218001419}. Best is trial 0 with value: 0.3311845286059629.
[I 2025-08-07 14:37:17,777] Trial 1 finished with value: 0.3432715551974214 and parameters: {'n_estimators': 100, 'max_depth': 25, 'min_samples_leaf': 380, 'min_impurity_decrease': 0.003424802669483067}. Best is trial 1 with value: 0.3432715551974214.
[I 2025-08-07 14:42:18,879] Trial 2 finished with value: 0.3295729250604351 and parameters: {'n_estimators': 250, 'max_depth': 45, 'min_samples_leaf': 220, 'min_impurity_decrease': 0.012867427637255695}. Best is trial 1 with value: 0.3432715551974214.
[I 2025-08-07 14:48:17,420] Trial 3 finished with value: 0.34286865431103947 and parameters: {'n_estimators': 200, 'max_depth': 35, 'min_samples_leaf':

In [60]:
param_dict = study_rf.best_params
param_dict.update({'criterion': 'log_loss', 'random_state': rs})

In [62]:
param_dict

{'n_estimators': 100,
 'max_depth': 25,
 'min_samples_leaf': 380,
 'min_impurity_decrease': 0.003424802669483067,
 'criterion': 'log_loss',
 'random_state': 42}

In [51]:
optuna.visualization.plot_optimization_history(study_rf, target_name='recall @ 5\% fpr')

Ну здесь увидел маленькие показатели, на уровне той же логистической регрессии, которая тратит на порядок меньше ресурсов. Решил, что неправильно выбрал данные, поэтому обучим небольшой RandomForest на наших данных, и возьмем лучшие 32 признака из них:

In [ ]:
X, y = data.drop('fraud_bool', axis=1), data['fraud_bool']
X_f_train, X_f_test, y_f_train, y_f_test = train_test_split(X, y, test_size=0.25, random_state=rs)

f_selection_model = ensemble.RandomForestClassifier(
    **{'n_estimators': 150,
    'max_depth': 45,
    'min_samples_leaf': 380,
    'min_impurity_decrease': 0.00017,
    'criterion': 'log_loss',
    'random_state': 42}
)
f_selection_model.fit(X_f_train, y_f_train)

rf_pred = f_selection_model.predict_proba(X_f_test)
print('Recall for Random Forest: {:.2f} %'.format(get_recall(y_f_test, rf_pred)*100))
print('\nFeature importances:\n\n', f_selection_model.feature_importances_)

Recall for Random Forest: 35.97 %

Feature importances:

 [4.26682264e-02 1.23400096e-02 2.32082753e-02 0.00000000e+00
 0.00000000e+00 5.55006429e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.72198213e-02
 2.52723820e-03 2.25536288e-02 7.59183807e-04 1.28292416e-03
 2.54872539e-02 7.66525461e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.60470447e-02 2.06719219e-03 0.00000000e+00
 4.73702718e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.25378010e-01
 1.55536099e-01 1.90939700e-03 6.07694098e-03 1.36092942e-04
 1.28648124e-03 9.63962784e-02 4.47688155e-02 4.31532429e-03
 4.29418205e-03 4.38844773e-03 6.66540535e-02 2.10969879e-02
 1.10557974e-01 7.70697311e-03 1.20967962e-02 0.00000000e+00
 1.16706196e-02 9.17311896e-03 3.03553988e-04 2.58123927e-02
 1.07471560e-02 0.00000000e+00 0.00000000e+00 1.11097255e-03
 0.00000000e+00 2.85077237e-04 8.78505363e-03 0.00000000e+00
 0.00000000e+00 4.54747135e

In [26]:
del(X_f_train, X_f_test, y_f_train, y_f_test)

NameError: name 'X_f_train' is not defined

In [68]:
results_dict = {f_selection_model.feature_names_in_[key]: f_selection_model.feature_importances_[key] for key in range(len(f_selection_model.feature_names_in_)) if f_selection_model.feature_importances_[key] > 0.001}
results_dict

{'income': 0.04266822644850516,
 'name_email_similarity': 0.012340009592618694,
 'current_address_months_count': 0.02320827529406699,
 'intended_balcon_amount': 0.0055500642884294595,
 'credit_risk_score': 0.037219821303250814,
 'email_is_free': 0.0025272381998924466,
 'phone_home_valid': 0.022553628768104548,
 'bank_months_count': 0.0012829241551066911,
 'has_other_cards': 0.02548725386339376,
 'proposed_credit_limit': 0.007665254607758069,
 'keep_alive_session': 0.02604704473129529,
 'device_distinct_emails_8w': 0.0020671921905482367,
 'only_one_valid': 0.004737027181686091,
 'housing_status_BA': 0.1253780104341245,
 'CA_BA': 0.15553609893159373,
 'housing_status_BE': 0.001909396997933975,
 'CA_AC': 0.006076940982867111,
 'payment_type_AC': 0.001286481242925689,
 'device_os_windows': 0.09639627842819029,
 'windows_AC': 0.04476881552149621,
 'device_os_linux': 0.00431532429164111,
 'device_os_other': 0.004294182053633944,
 'windows_AB': 0.004388447729553166,
 'AC_BA': 0.06665405352617

In [72]:
len(results_dict)

42

In [71]:
[key for key in results_dict.keys() if key not in df.columns]

['name_email_similarity',
 'current_address_months_count',
 'email_is_free',
 'bank_months_count',
 'keep_alive_session',
 'CA_AC',
 'payment_type_AC',
 'windows_AC',
 'windows_AB',
 'AC_BA',
 'AB_BA',
 'n_e_cat_similarity',
 'cur_address_months_binned_first_spike',
 'cur_address_fraud_zone',
 'cur_address_months_binned_end_spike',
 'is_credit_limit_high',
 'new_email_entrance']

Видим примерно те же результаты, хотя чуть больше признаков. 

## GradientBoosting

In [ ]:
param_grid_gb = {
    'n_estimators': (100, 300, 50),
    'max_depth': (5, 12, 1),
    'num_leaves': (20, 100, 10),
    'reg_alpha': (1e-3, 1.0),
    'reg_lambda': (1e-3, 1.0),
    'min_gain_to_split': (0.0, 0.1),
    'min_data_in_leaf': (20, 100, 10),
    'class_weight': 'balanced',
    'random_state': rs,
}

gb_optuna = make_optuna_objective(
    model=lgbm.LGBMClassifier,
    param_grid=param_grid_gb,
    X_train=X_train_resampled,
    y_train=y_train_resampled,
    X_val=X_val,
    y_val=y_val
)


study_gb = optuna.create_study(study_name='Gradient Boosting (lgbm)', direction='maximize')
study_gb.optimize(gb_optuna, n_trials=25)

In [30]:
optuna.visualization.plot_optimization_history(study_gb, target_name='recall @ 5\% fpr')

In [31]:
study_gb.best_params

{'n_estimators': 300,
 'max_depth': 12,
 'num_leaves': 80,
 'reg_alpha': 0.1997799598953282,
 'reg_lambda': 0.9936086042807553,
 'min_gain_to_split': 0.03708299877911875,
 'min_data_in_leaf': 70}

In [ ]:
param_grid_gb = {
    'n_estimators': (400, 600, 50),
    'max_depth': (10, 16, 2),
    'num_leaves': (60, 150, 20),
    'reg_alpha': (0.2, 2.2),
    'reg_lambda': (0.5, 2.2),
    'min_gain_to_split': (0.03, 0.1),
    'min_data_in_leaf': (70, 200, 40),
    'class_weight': 'balanced',
    'random_state': rs,
    'verbose': -1
}

gb_optuna = make_optuna_objective(
    model=lgbm.LGBMClassifier,
    param_grid=param_grid_gb,
    X_train=X_train_resampled,
    y_train=y_train_resampled,
    X_val=X_val,
    y_val=y_val
)


study_gb_2 = optuna.create_study(study_name='Gradient Boosting (lgbm)', direction='maximize')
study_gb_2.optimize(gb_optuna, n_trials=25)

[I 2025-08-07 17:49:53,730] A new study created in memory with name: Gradient Boosting (lgbm)
[I 2025-08-07 17:50:14,122] Trial 0 finished with value: 0.37107171635777597 and parameters: {'n_estimators': 500, 'max_depth': 12, 'num_leaves': 60, 'reg_alpha': 0.756483535908177, 'reg_lambda': 0.7190505609180924, 'min_gain_to_split': 0.030476974600283195, 'min_data_in_leaf': 110}. Best is trial 0 with value: 0.37107171635777597.
[I 2025-08-07 17:50:29,722] Trial 1 finished with value: 0.37308622078968573 and parameters: {'n_estimators': 350, 'max_depth': 12, 'num_leaves': 60, 'reg_alpha': 0.3500147376075564, 'reg_lambda': 0.7634494709412958, 'min_gain_to_split': 0.08515746922093015, 'min_data_in_leaf': 190}. Best is trial 1 with value: 0.37308622078968573.
[I 2025-08-07 17:50:48,844] Trial 2 finished with value: 0.3726833199033038 and parameters: {'n_estimators': 450, 'max_depth': 14, 'num_leaves': 60, 'reg_alpha': 0.38362074966190013, 'reg_lambda': 0.7462407960838964, 'min_gain_to_split': 

In [49]:
gb_best_params = study_gb_2.best_params
gb_best_params.update({'class_weight': 'balanced',
    'random_state': rs,
    'verbose': -1})

In [ ]:
gb_clf = lgbm.LGBMClassifier(**gb_best_params)
gb_clf.fit(X_train_resampled, y_train_resampled)

gb_pred = gb_clf.predict_proba(X_val)
print('Recall for Random Forest: {:.2f} %'.format(get_recall(y_val, gb_pred)*100))


Recall for Random Forest: 36.06 %


Итак, пока что градиентный бустинг показал себя лучше себя. Но это был градиентный бустинг из lgbm. Попробуем catboost от Яндекса:

*P.S. оказалось, что якобы линейный датасет показывает себя лучше предназначенного для деревьев. Поэтому оставили его.*

*P.P.S. решил попробовать отобранные с помощью SelectKBset признаки, результат упал на пару процентов.*

In [ ]:
# X_train_k, X_val_k, y_train_k, y_val_k = train_test_split(data[kbest_features], data['fraud_bool'], 
#                                                   test_size=0.3,
#                                                   stratify=data['fraud_bool'], 
#                                                   random_state=rs)

In [23]:
# def optuna_optimize_catboost(trial):
#     """
#     Функция подбора гиперпараметров для CatBoost
#     """
#     params = {
#         'iterations': trial.suggest_int('iterations', 400, 500, 100),             
#         'depth': trial.suggest_int('depth', 4, 14, 4),                           
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15),               
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.5, 10),                     
#         'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),                       
#         'border_count': trial.suggest_int('border_count', 32, 254, 50),                  
#         'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),  
#         'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),          
#         'class_weights': [1, 90],                
#         'random_state': rs,
#         'verbose': 0
#     }
    
#     if params['bootstrap_type'] == 'Bayesian':
#         params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0.0, 1.0)
        
    
#     model = cat.CatBoostClassifier(**params)

#     model.fit(
#         X_train_k, y_train_k,
#         eval_set=(X_val_k, y_val_k),
#         early_stopping_rounds=50,
#         verbose=0
#     )

#     y_pred_proba = model.predict_proba(X_val_k)
    
#     score = get_recall(y_val_k, y_pred_proba)
#     return score

# study_cat = optuna.create_study(study_name='CatBoost', direction="maximize")
# study_cat.optimize(optuna_optimize_catboost, n_trials=50)

In [38]:
def optuna_optimize_catboost(trial):
    """
    Функция подбора гиперпараметров для CatBoost
    """
    params = {
        'iterations': trial.suggest_int('iterations', 400, 500, 100),             
        'depth': trial.suggest_int('depth', 4, 14, 4),                           
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15),               
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.5, 10),                     
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),                       
        'border_count': trial.suggest_int('border_count', 32, 254, 50),                  
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),  
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),          
        'class_weights': [1, 90],                
        'random_state': rs,
        'verbose': 0
    }
    
    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0.0, 1.0)
        
    
    model = cat.CatBoostClassifier(**params)

    model.fit(
        X_train_lin, y_train_lin,
        eval_set=(X_val_lin, y_val_lin),
        early_stopping_rounds=50,
        verbose=0
    )

    y_pred_proba = model.predict_proba(X_val_lin)
    
    score = get_recall(y_val_lin, y_pred_proba)
    return score

study_cat = optuna.create_study(study_name='CatBoost', direction="maximize")
study_cat.optimize(optuna_optimize_catboost, n_trials=50)

[I 2025-08-10 15:11:56,558] A new study created in memory with name: CatBoost
[I 2025-08-10 15:12:03,646] Trial 0 finished with value: 0.44721998388396456 and parameters: {'iterations': 500, 'depth': 8, 'learning_rate': 0.1027130600814946, 'l2_leaf_reg': 9.62838556031353, 'random_strength': 0.04336716031433763, 'border_count': 82, 'grow_policy': 'SymmetricTree', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.44721998388396456.
[I 2025-08-10 15:12:22,566] Trial 1 finished with value: 0.4564867042707494 and parameters: {'iterations': 500, 'depth': 4, 'learning_rate': 0.08248735310429404, 'l2_leaf_reg': 9.782873748106118, 'random_strength': 0.933762866293711, 'border_count': 182, 'grow_policy': 'Depthwise', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.9780925002666725}. Best is trial 1 with value: 0.4564867042707494.
[I 2025-08-10 15:12:34,032] Trial 2 finished with value: 0.43110394842868655 and parameters: {'iterations': 400, 'depth': 8, 'learning_rate': 0.08038023912

Ну и абсолютный победитель - CatBoost, показывающий показатель 45-46%. Причем примечательно, что модель показывает результат лучше на "линейных" отобранных признаках.

In [39]:
best_params = study_cat.best_params
best_params.update({'class_weights': [1, 9], 'random_state': rs, 'verbose': 0})

cat_clf = cat.CatBoostClassifier(**best_params)
cat_clf.fit(
    X_train_lin, y_train_lin,
    eval_set=(X_val_lin, y_val_lin),
    early_stopping_rounds=50,
    verbose=0
)

In [ ]:
save_model(cat_clf, 'cat_boost')

Осталось проверить нейронные сети. Для этого перешел в другой jupyter notebook для подключения к wsl и работы с tensorflow.keras. 

---

---

# 7. Итоги и Выбор модели

### Завершен этап исследований и экспериментов. 

* Проведен полный, тщательный, разносторонний анализ данных
* Выявлены закономерности между признаками и целевой переменной
* Спроектированы и отобраны разными способами и методами фичи
* Протестированы модели на разных данных, с различными гиперпараметрами, подбор осуществлялся в том числе современными методами, типо Optuna. Среди моделей весь спектр, применимый к данному кейсу - от простешей логистической регресии до нейронных сетей.
* Использовалась методология Agile - с переходами в том числе на предыдущие этапы для корректировки полученных ранее результатов.

В общем, если коротко - **колоссальная работа**. 
<br>Лучший результат по метрику "recall при 5% fpr" на валидации - 46%.
<br>Модель - CatBoost (Градиентный бустинг).
<br><br>Вроде бы не очень высокий результат, но интерпретируется он так: если мы допускаем ошибку всего 5% для немошеннических транзакций, то мошеннические транзакции правильно угадываются с вероятностью 46%. Учитывая относительно небольшой размер данных, это достаточный показатель. Конечно, нужно еще провести замер на тесте, но сперва нужно построить пайплайн. <br><br>
### Какие преимущества у этого проекта:
1. Проведение полного цикла разработки: от аналитики данных до построения моделей и подбора гиперпараметров; использование современной технологии Agile - а значит, более продуманный и "крепкий" результат
2. Работа с данными: от восстановления данных с помощью модели, проведения статистических тестов до фича-инжиниринга с продуманным отбором признаков
3. Перебор множества разных моделей задачи классификации, использование нейронных сетей и т.д.
4. Использование продвинутых технологий машинного обучения: lgbm, catboost, keras
5. Четкая стуктура проекта: легко ориентироваться в папках, работа выполнена в jupyter notebook для комментариев, пояснений и выводов, какие-то общие функции вынесены в utils.support.py. <br><br>
### Какие сложности возникли:
1. Специфичность данных - а именно, что одни признаки были анонимные, другие - уже масштабированные/измененные так же в  связи с конфиденциальностью данных. Наверное, это было самое сложное в этой работе. Например, пришлось перебирать комбинации признаков типа "статус резидента" или "тип карты", поскольку нельзя было логически выдвинуть каких-либо кандидатов. По итогу, они показывали наибольшую информативность.
2. Большое количество признаков. Да, 32 признака звучит не так серьезно, особенно для нейронки, но учитывая подход к этой задаче - а именно решение с помощью классического ML - пришлось изучать все признаки по одному, где-то преобразовывать Боксом-Коксом, где-то квантильным преобразованием, где-то находить связи и т.д. Можно увидеть по размеру main.ipynb, что фича инжиниринг занял где-то 60% времени проекта.
3. Выбор признаков. Довольно трудно было решиться, какие признаки выбирать: разные методы показывали разные результаты. Подробно о способах, их плюсах и минусах написано в разделе 3 "Отбор признаков". И вот, казалось бы, математически, например, для деревьев больше подходили признаки с наибольшим значение mutual_info или SHAP алгоритма, но нет - подошли лидеры по линейным характеристикам, таким, как pointbiserial (хотя каждом топе где-то 10-14 признаков оставились теми же.). Но в любом случае, SelectKBest справился хуже, чем я вручную.
4. Дисбаланс данных. Я читал кучу статей на эту тему, но, как говорится, одно дело теория, а другое - практика. В моих данных был всего 1% мошеннических атак, тренировочная выборка всего на 750.000 семплов, поэтому нужно было или учитывать веса классов, или применять оверсемплинг. Пробовал и SMOTE, и SVMSMOTE (SMOTE на основе методов опорных векторов), и ADASYN - надеялся больше всего на последний вариант, т.к. на распределениях увидел, что в разрезе фрод и нефрод несильно отличаются - с разными параметрами количества ближайших соседей, но результат один - ухудшение метрики по сравнению даже с обычным train. Поэтому пришлось просто применять веса к классам, но опять же - недостаточно данных на фрод, от этого и 
5. GridSearch - не работал. Хотел делать сразу подбор гиперпараметров к моделям, хотя бы небольшой, вдруг где-то бы прирост был +10%. Но на cv были нулевые показатели метрики, поэтому grid_search проcто возвращал последнюю модель. Насколько я понял, на cv был слишком низкий recall для fpr=5%, а также оверсемплинг добавлял слишком много бесполезного шума. Итого пришлось вручную подбирать параметры, на что ушло время.
<br><br>
### Что и как можно улучшить:
1. Низкий показатель - конечно, эталлоного нет, но хотелось бы, чтобы он был повыше. Причины этого - выше.
2. Возможно, неинформативность отобранных признаков. Т.е. нужно вернуться к фича-инжинирингу - посмотреть на другие комбинации данных. К сожалению, здесь не получится, например, добавить еще данных из источников из-за анонимности и масштабированности, но попробовать посмотреть на разные отношения или комбинации признаков еще можно.



In [128]:
X_train_lin.shape

(525000, 32)

Наконец - создаем пайплайн и проверяем на тестовых данных.

# 8. Подготовка сервиса: pipeline

In [ ]:
# загружаем данные
lin_features = open_list('logs/top_linear_features.txt')

data = pd.read_csv('data/bank_data.csv', sep=',')
data_test = open_data('data/bank_test.csv')

best_model = open_model('logs/cat_boost.pkl')
best_params = best_model.get_params()

In [146]:
X_val_lin.index

Index([945302, 311914, 123776, 652637,  26345, 849175,  62615, 442473, 999736,
       636245,
       ...
       377454, 617114,  63189, 249570, 671176, 426148, 463462, 994980, 492257,
       987433],
      dtype='int64', length=225000)

In [ ]:
train_inds = data.index.difference(data_test.index)
data_train = data.loc[train_inds]
data_test = data.loc[data_test.index]

In [148]:
# data_train = data.iloc[[ind for ind in data.index if ind not in data_test.index and ind not in X_val.index]]
# data_test = data.iloc[[ind for ind in data.index if ind in data_test.index]]

In [149]:
data_test.shape

(250000, 32)

In [150]:
best_params

{'iterations': 500,
 'learning_rate': 0.1128707786736137,
 'depth': 4,
 'l2_leaf_reg': 3.4780256980614572,
 'border_count': 82,
 'verbose': 0,
 'class_weights': [1, 9],
 'random_strength': 0.6177624200217992,
 'bagging_temperature': 0.9754464027374743,
 'bootstrap_type': 'Bayesian',
 'random_state': 42,
 'grow_policy': 'Lossguide'}

In [151]:
X_test, y_test = data_test.drop('fraud_bool', axis=1), data_test['fraud_bool']
X_train, y_train = data_train.drop('fraud_bool', axis=1), data_train['fraud_bool']

In [152]:
from sklearn.base import BaseEstimator, TransformerMixin

class MissingValuesPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.medians_ = {}

    def fit(self, X, y=None):
        df = X.copy()
        self.medians_['current_address_months_count'] = df.loc[df['current_address_months_count'] != -1, 
                                                               'current_address_months_count'].median()
        self.medians_['session_length_in_minutes'] = df.loc[df['session_length_in_minutes'] != -1, 
                                                            'session_length_in_minutes'].median()

        return self

    def transform(self, X):
        df = X.copy()
        df['source'] = (df['source'] == 'INTERNET').astype(int)
        df.drop('prev_address_months_count', axis=1, inplace=True)

        df['current_address_months_count'].replace(-1, self.medians_['current_address_months_count'], inplace=True)
        df['session_length_in_minutes'].replace(-1, self.medians_['session_length_in_minutes'], inplace=True)
        df['device_distinct_emails_8w'].replace(-1, 1, inplace=True)

        return df
    
    
    
    
class BankMonthsInputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = ensemble.RandomForestRegressor(n_estimators=40, 
                                                        criterion='friedman_mse', 
                                                        max_depth=8,
                                                        min_samples_split=25000
                                                        )
        self.drop_list = None
        
    def fit(self, X, y=None):
        train_df = X.copy()
        cat_list = [x for x in train_df.columns if train_df[x].dtype == 'object']
        train_df = train_df[train_df['bank_months_count'] != -1]
        self.drop_list = ['bank_months_count', 'device_fraud_count'] + cat_list
        
        X_train, y_train = train_df.drop(self.drop_list, axis=1), train_df['bank_months_count']
        self.model.fit(X_train, y_train)
        return self
    
    def transform(self, X):
        df = X.copy()
        X_test = df[df['bank_months_count'] == -1]
        
        if self.drop_list is None:
            raise ValueError('Must use fit on train data first!')
        
        if not X_test.empty:
            X_test.drop(self.drop_list, axis=1, inplace=True)
            pred = self.model.predict(X_test)
            df.loc[df['bank_months_count'] == -1, 'bank_months_count'] = pred
        
        return df




class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self, features_list):
        self.features_list = features_list
        self.income_map = {}
        self.quantile_similarity = None
        self.quantile_bank_months = None
        self.power_transformer = None
        self.pca_velocity = None
        # список признаков, связанных со скоростями, которые нужно разложить на главные компоненты
        self.vel_features = [
            'velocity_6h', 'velocity_24h', 'velocity_4w',
            'velocity_day_change', 'velocity_term_month_change',
            'velocity_day_ratio', 'velocity_term_month_ratio'
        ]
        self.bank_branch_bins = None

    # функция, кодирующие порядковый признак по возрастанию процента мошеннических атак
    def _get_fraud_ranking(self, X, y, feature):
        df = X.copy()
        
        df['fraud_bool'] = y
        fraud_percentage_month = df.groupby(feature)['fraud_bool'].mean()
        # сортируем и ранжируем по процентам фрод по месяцам
        fraud_percentage_month = fraud_percentage_month.sort_values().rank()
        # заполняем пропуски
        fraud_percentage_month = fraud_percentage_month.fillna(0).astype('uint8') - 1
        
        return fraud_percentage_month.to_dict()

        
    def fit(self, X, y):
        df = X.copy()

        df['velocity_day_change'] = df['velocity_6h'] - df['velocity_24h']
        df['velocity_term_month_change'] = df['velocity_6h'] - df['velocity_4w']
        df['velocity_day_ratio'] = df['velocity_6h']/df['velocity_24h']
        df['velocity_term_month_ratio'] = df['velocity_6h']/df['velocity_4w']
        
        self.income_map = self._get_fraud_ranking(X, y, 'income')
        
        self.quantile_similarity = preprocessing.QuantileTransformer(output_distribution='normal')
        self.quantile_similarity.fit(df[['name_email_similarity']])
        
        self.quantile_bank_months = preprocessing.QuantileTransformer(output_distribution='normal')
        self.quantile_bank_months.fit(df[['bank_months_count']])
        
        self.power_transformer = preprocessing.PowerTransformer(method='yeo-johnson')
        self.power_transformer.fit(df[['current_address_months_count']])

        self.pca_velocity = decomposition.PCA(n_components=4, svd_solver='arpack')
        self.pca_velocity.fit(df[self.vel_features])
        
        # считаем процентили, чтобы гарантированно получить 4 бина
        bins = np.percentile(df['bank_branch_count_8w'], [0, 25, 50, 75, 100])
        # убираем повторяющиеся границы (если они есть)
        self.bank_branch_bins = np.unique(bins)
        
        return self


    def transform(self, X):
        df = X.copy()
        # создаем фичи, связанные со скоростями
        df['velocity_day_change'] = df['velocity_6h'] - df['velocity_24h']
        df['velocity_term_month_change'] = df['velocity_6h'] - df['velocity_4w']
        df['velocity_day_ratio'] = df['velocity_6h']/df['velocity_24h']
        df['velocity_term_month_ratio'] = df['velocity_6h']/df['velocity_4w']
        
        # бинарные признаки-флаги
        df['only_one_valid'] = df['phone_home_valid'] ^ df['phone_mobile_valid']
        
        df['housing_status_BA'] = (df['housing_status'] == 'BA').astype('uint8')
        df['housing_status_BE'] = (df['housing_status'] == 'BE').astype('uint8')
        df['payment_type_AA'] = (df['payment_type'] == 'AA').astype('uint8')
        df['device_os_windows'] = (df['device_os'] == 'windows').astype('uint8')
        df['device_os_linux'] = (df['device_os'] == 'linux').astype('uint8')
        df['device_os_other'] = (df['device_os'] == 'other').astype('uint8')
        
        df['CA_BA'] = ((X['employment_status']=='CA') & (X['housing_status']=='BA')).astype('uint8')
        df['BA_windows'] = ((X['housing_status']=='BA') & (df['device_os'] == 'windows')).astype('uint8')
        
        df['intended_balcon_peak_1'] = ((df['intended_balcon_amount'] > 0 )
                                  & (df['intended_balcon_amount'] < 60)).astype('uint8')
        df['cur_address_months_binned_danger_spike'] = ((df['current_address_months_count'] > 25) &
                                                          (df['current_address_months_count'] <= 200))\
                                                              .astype('uint8') 
                                                              
        # рекодируем income по train
        df['income'] = df['income'].map(self.income_map).fillna(0).astype('uint8')
        
        # преобразовываем величины
        df['n_e_norm_similarity'] = self.quantile_similarity.transform(df[['name_email_similarity']])
        df['bank_months_count_qt'] = self.quantile_bank_months.transform(df[['bank_months_count']])
        df['cur_address'] = self.power_transformer.transform(df[['current_address_months_count']])
        df['zip_logged'] = np.log(df['zip_count_4w']+1)
        df['birth_distinct_emails_logged'] = np.log(df['date_of_birth_distinct_emails_4w']+1)
        
        # находим пиковое значение bank_branch_count
        labels = [f"q{i+1}" for i in range(len(self.bank_branch_bins) - 1)]
        
        bank_branch_count_q = pd.cut(X['bank_branch_count_8w'], bins=self.bank_branch_bins, 
                                        labels=labels, include_lowest=True)
        bank_branch_frame = pd.get_dummies(bank_branch_count_q).astype('uint8')

        q1 = bank_branch_frame.get('q1', pd.Series(0, index=df.index))
        q3 = bank_branch_frame.get('q3', pd.Series(0, index=df.index))
        
        if not q1.any() or q3.any():
            print('WARNING: слипшиеся квантили для bank_branch_count_8w')
        # создаем бинарный индикатор пикового значения
        df['bank_branch_peak'] = (q1 | q3).astype('uint8')
        
        # создаем фичи-формулы
        df['max_account_sum'] = df['bank_months_count'] * df['proposed_credit_limit']
        df['zip_transaction_ratio'] = df['zip_count_4w'] * 2 / (df['bank_branch_count_8w']+1)
        df['max_zip_sum'] = df['zip_count_4w'] * df['proposed_credit_limit']
        df['zip_sum_ratio'] = df['zip_count_4w'] / df['proposed_credit_limit']
        
        # берем две главные компоненты признаков-скоростей
        transformed_vel = self.pca_velocity.transform(df[self.vel_features])
        df['vel_1'], df['vel_2'] = transformed_vel[:, 0], transformed_vel[:, 1]
    
        # возвращаем итоговый датафрейм
        return df[self.features_list]

In [153]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('MissingValuesPreprocessor', MissingValuesPreprocessor()),
    ('BankMonthsInputer', BankMonthsInputer()),
    ('FeatureEngineering', FeatureEngineering(lin_features)),
    ('CatBoost', cat.CatBoostClassifier(**best_params))
])

pipe.fit(X_train, y_train)

y_pred_train = pipe.predict_proba(X_train)
y_pred_test = pipe.predict_proba(X_test)

print('Recall@FPR=5 for train: {:.2f} %'.format(get_recall(y_train, y_pred_train)*100))
print('Recall@FPR=5 for test: {:.2f} %'.format(get_recall(y_test, y_pred_test)*100))

Recall@FPR=5 for train: 58.31 %
Recall@FPR=5 for test: 48.28 %


Здесь попробовал обучить пайплайн как на обычных train данных (т.е. не включая валидационную выборку), так и на полном train. Для первого результат был:

Recall@FPR=5 for train: 58.38 %

Recall@FPR=5 for test: 47.77 %

Для второго, как видно выше - результат лучше.
Итоговый, лучший результат по метрике - 
<br><br>
Recall@FPR=5 for train: 58.31 %<br>
Recall@FPR=5 for test: 48.28 %

In [165]:
save_model(pipe, 'final_pipeline')

In [1]:
%pip freeze

absl-py==2.2.1Note: you may need to restart the kernel to use updated packages.

aiohappyeyeballs==2.6.1
aiohttp==3.11.16
aiosignal==1.3.2
alembic==1.13.2
annotated-types==0.7.0
anyio==4.10.0
arch==7.2.0
asttokens==2.4.1
astunparse==1.6.3
attrs==23.2.0
bayesian_changepoint_detection==0.2.dev1
beautifulsoup4==4.12.3
bokeh==3.6.0
Brotli==1.1.0
captum==0.8.0
catboost==1.2.5
category-encoders==2.6.3
certifi==2024.2.2
chardet==5.2.0
charset-normalizer==3.3.2
clarabel==0.9.0
click==8.1.7
cloudpickle==3.0.0
clustergram==0.8.1
cmdstanpy==1.2.5
colorama==0.4.6
coloredlogs==15.0.1
colorlog==6.8.2
comet-ml==3.43.2
comm==0.2.2
configobj==5.0.8
contourpy==1.3.2
country-converter==1.2
cvxpy==1.6.0
cycler==0.12.1
Cython==3.0.12
dacite==1.8.1
dash==2.17.1
dash-bootstrap-components==1.3.1
dash-core-components==2.0.0
dash-html-components==2.0.0
dash-table==5.0.0
dash_colorscales==0.0.4
dash_daq==0.5.0
debugpy==1.8.1
decorator==5.1.1
dtale==3.12.0
dulwich==0.22.1
et-xmlfile==1.1.0
everett==3.1.0
executin

---

Ну, наконец, последний, бонусный пункт - это любительский, небольшой сервер для предсказания фрода по какой-то строке, или даже целому датафрейму на FastAPI. Для этого достаточно установить зависимости (желательно в отдельной среде) и запустить приложение app.py.